<a href="https://colab.research.google.com/github/Nischalj10/gpt2/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#get the dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-04-10 11:52:02--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2024-04-10 11:52:02 (49.4 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
# reading the data
with open('input.txt',  'r', encoding='utf-8') as f:
  text = f.read()

In [3]:
print("length of dataset in chats", len(text))

length of dataset in chats 1115394


In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder - take a string an output list of integers
decode = lambda l: ''.join([itos[i] for i in l]) #decoder - take a list of integers and output text

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8
data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size+1]
y = train_data[1:block_size+1]

for t in range(block_size):
  context = x[:t+1]
  target = y[t]
  print(f'when input is  {context} - output is {target}')

when input is tensor([18]) - output is 47
when input is tensor([18, 47]) - output is 56
when input is tensor([18, 47, 56]) - output is 57
when input is tensor([18, 47, 56, 57]) - output is 58
when input is tensor([18, 47, 56, 57, 58]) - output is 1
when input is tensor([18, 47, 56, 57, 58,  1]) - output is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) - output is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) - output is 58


In [17]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size, ))
  x = torch.stack([data[i : i + block_size] for i in ix])
  y = torch.stack([data[i+1 : i + block_size + 1] for i in ix])
  return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-------')

for b in range(batch_size): #batch dimension
  for t in range(block_size): # time dimension
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f'when input is  {context} - output is {target}')

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------
when input is  tensor([24]) - output is 43
when input is  tensor([24, 43]) - output is 58
when input is  tensor([24, 43, 58]) - output is 5
when input is  tensor([24, 43, 58,  5]) - output is 57
when input is  tensor([24, 43, 58,  5, 57]) - output is 1
when input is  tensor([24, 43, 58,  5, 57,  1]) - output is 46
when input is  tensor([24, 43, 58,  5, 57,  1, 46]) - output is 43
when input is  tensor([24, 43, 58,  5, 57,  1, 46, 43]) - output is 39
when input is  tensor([44]) - output is 53
when input is  tensor([44, 53]) - output is 56
when input is  tensor([44, 53, 56]) - output is 1
when 

In [28]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    #each token directly reads off the logits for the next token from a lookup table
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    # idx and targets are both (B,T) tensor of integers
    logits = self.token_embedding_table(idx)

    if targets is None:
      loss = None
    else :
      #loss = F.cross_entropy(logits, targets) # this won't work as pytorch expets in the form of B, C, T and we have logits in the form of B, T, C
      B, T, C = logits.shape
      logits = logits.view(B*T, C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):
    #idx is (B,T) array of indices in the current context
    for _ in range(max_new_tokens):
      #get the predictions
      logits ,loss = self(idx)
      #focus only on the last time step
      logits = logits[:, -1, :] # becomes (B,C) # taking only the last character to predict the next one as we are making a bigram model
      #apply softmax to get probs
      probs = F.softmax(logits, dim=-1) #(B,C)
      #get the prediction from the sample
      idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
      # append sampled index to the running sequence
      idx = torch.cat((idx, idx_next), dim=1) #(B, T+1)
    return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist())) #sending in the newline character (a 1*1 input tensor containing zero (batch size = 1 and input/ T = 1 as well containing 0))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [29]:
#create a pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) #typical is 3e-4 - but this 1e-3 high rate would work for our small nn

In [38]:
batch_size = 32
for steps in range(10000):
  #sample a batch of data
  xb, yb = get_batch('train')

  #evaluate the loss
  logits, loss = m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())

2.4010491371154785


In [44]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=300)[0].tolist()))


Medotrepe th htome be rt Foue
A:
Nouthiso t the tposind mm, t;
BYed e t acacet dis mpl arcern yot te morden-
O'd nout shik.
Mare? youlllabepllithr I fes angrvonk t! ikmeerecowie, htomy w me be porowalo hen fof wis chare oumounde, hen bt t mpad sshead mandowilifunthays bugmediestyoureal ighitsoanoulo


In [2]:
import torch
#toying with attention
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.rand(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [6]:
xbow = torch.zeros(B,T,C)

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] #shape = t, C - taking all the prevoius t elements and mean of them
        xbow[b,t] = torch.mean(xprev, 0)

print(x[0])
xbow[0]

tensor([[0.0783, 0.4956],
        [0.6231, 0.4224],
        [0.2004, 0.0287],
        [0.5851, 0.6967],
        [0.1761, 0.2595],
        [0.7086, 0.5809],
        [0.0574, 0.7669],
        [0.8778, 0.2434]])


tensor([[0.0783, 0.4956],
        [0.3507, 0.4590],
        [0.3006, 0.3156],
        [0.3717, 0.4108],
        [0.3326, 0.3806],
        [0.3953, 0.4140],
        [0.3470, 0.4644],
        [0.4134, 0.4368]])

In [10]:
wei = torch.tril(torch.ones(T,T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (T, T) @ (B, T, C) -> (B, T, T) @ (B, T, C) 
torch.allclose(xbow, xbow2)

True

In [4]:
import torch.nn.functional as F
#version 3
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T,T) #affinity starting with zero
wei = wei.masked_fill(tril == 0, float('-inf')) # past can't communicate with future
wei = F.softmax(wei, dim=-1) # and -inf elements will become zero and the rest will become 1. then the softmax divide term will avg and give the same kind of wei as above

xbow3 = wei @ x
torch.allclose(xbow3, xbow)

NameError: name 'xbow' is not defined

In [15]:
import torch
import torch.nn as nn
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

#single attention head
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # B, T, 16
q = query(x) # B, T, 16
v = value(x) # B, T, 16

wei = q @ k.transpose(-2, -1) * head_size**-0.5 #transposing the last two dimensions -> B, T, 16 @ B, 16, T = B, T, T


tril = torch.tril(torch.ones(T,T))
# wei = torch.zeros(T,T)
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = torch.softmax(wei, dim=-1)

# out = wei @ x 
out = wei @ v
out.shape

torch.Size([4, 8, 16])

In [16]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3966, 0.6034, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3069, 0.2892, 0.4039, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3233, 0.2175, 0.2443, 0.2149, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1479, 0.2034, 0.1663, 0.1455, 0.3369, 0.0000, 0.0000, 0.0000],
        [0.1259, 0.2490, 0.1324, 0.1062, 0.3141, 0.0724, 0.0000, 0.0000],
        [0.1598, 0.1990, 0.1140, 0.1125, 0.1418, 0.1669, 0.1061, 0.0000],
        [0.0845, 0.1197, 0.1078, 0.1537, 0.1086, 0.1146, 0.1558, 0.1553]],
       grad_fn=<SelectBackward0>)

In [17]:
out[0]

tensor([[-1.5713e-01,  8.8009e-01,  1.6152e-01, -7.8239e-01, -1.4289e-01,
          7.4676e-01,  1.0068e-01, -5.2395e-01, -8.8726e-01,  1.9067e-01,
          1.7616e-01, -5.9426e-01, -4.8124e-01, -4.8599e-01,  2.8623e-01,
          5.7099e-01],
        [ 4.3974e-01, -1.4227e-01, -1.3157e-01,  2.8895e-03, -1.3222e-01,
          6.6093e-04, -2.7904e-01, -2.2676e-01, -2.8723e-01,  5.7456e-01,
          5.6053e-01, -2.5208e-01,  9.7243e-02,  1.0771e-01,  3.0455e-02,
          1.0727e+00],
        [ 4.3615e-01, -6.6358e-02, -2.9296e-01,  7.4315e-02,  5.4381e-02,
         -7.0388e-02, -6.8984e-02, -8.2153e-02, -2.9377e-01, -5.8952e-02,
          3.5887e-01, -2.3088e-03, -1.8212e-01, -3.6143e-02, -6.7189e-02,
          1.1412e+00],
        [ 4.2069e-01, -1.0619e-01, -2.9984e-01,  5.2820e-02,  2.0077e-01,
         -1.6048e-01, -3.5710e-02, -8.3110e-02, -1.7919e-01,  7.7992e-02,
          1.2719e-01,  2.2611e-02, -5.1811e-02,  7.4466e-02,  1.8131e-01,
          8.4463e-01],
        [ 3.9499e-01